In [5]:
import pymongo
import pandas as pd
import math 
from  qgrid import show_grid as show 
from datetime import datetime

client = pymongo.MongoClient('172.16.45.8', 27017)
db = client.rapls
sensors= db.sensor

In [6]:
def get_times(target,source=db.testcases):
    x= list(db.testcases.find({'name':target}))[0]
    x['execution']=datetime.utcfromtimestamp(int(x['execution']))
    x['begin']=datetime.utcfromtimestamp(int(x['begin']))
    x['end']=datetime.utcfromtimestamp(int(x['end']))
    x['warmup']=datetime.utcfromtimestamp(int(x['warmup']))
    return x 

def calculate_power(target , source=db.sensor):
    times=get_times(target)
    df = pd.DataFrame.from_dict(list(sensors.find({'timestamp' :{'$gte':times['begin'],'$lte':times['end']}})))
    result=df[df.target== target][["_id","rapl","timestamp"]]
    system=df[df.target== "system"] 
    result["rapl"]=result.apply(lambda row :math.ldexp(  system[system.timestamp==row['timestamp']].rapl.iloc[0]['0']['38']['RAPL_ENERGY_PKG'],-32), axis=1)
#     result['timestamp']= result.apply(lambda row :int(row['timestamp'].timestamp()),axis=1)
    return result

def calculate_energy(targetname) : 
    
    times=get_times(targetname)
    target=calculate_power(targetname)
    warmup = target[(target.timestamp >= times['warmup']) & (target.timestamp < times['execution']) ].loc[:,"rapl"].sum() 
    execution = target[(target.timestamp >= times['execution']) ].loc[:,"rapl"].sum() 
    return {'warmup':warmup,'execution': execution} 

In [8]:
calculate_energy('test13')

IndexError: list index out of range

In [20]:
pd.DataFrame.from_dict(list(db.testcases.find()))

,_id,begin,end,execution,name,warmup
0,5be18d8e25302356f78121ef,1541508355,1541508494,1541508418,tommtiv1int1,1541508381
